In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [30]:
def scrape_shein_products(url):
    driver = webdriver.Chrome()  # Replace with your browser driver (e.g., ChromeDriver)
    driver.get(url)
    
    # Wait for the pop-up window to load
    WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH,'//i[@class="iconfont icon-close she-close"]')))

    # Click button(close pop-up window)
    close_button = driver.find_element(By.XPATH,'//i[@class="iconfont icon-close she-close"]')
    close_button.click()
    
    # Scroll to bottom
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    # Wait for 10 seconds for the elements to load
    WebDriverWait(driver, 10)#.until(EC.presence_of_all_elements_located((By.XPATH, "//section[@class='S-product-item j-expose__product-item product-list__item']")))


    # Wait for the product list to load
    #WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'S-product-item j-expose__product-item product-list__item')))
    
    # Find all the elements with the specified XPath
    elements = driver.find_elements(By.XPATH, "//section[@role='listitem']")
    
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    
    products_data = []
    # Print the text inside each element
    for element in elements[0:10]:
        #print(element.text)
            
        WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'S-product-item__name')))

        # Extract product name, price, discount, average rating, tags, and top 100 reviews
        product_name = element.find_element(By.CLASS_NAME, 'S-product-item__name').text
        
        print("GOT " + product_name + "!")
        
        product_price = element.find_element(By.CLASS_NAME, 'product-item__camecase-wrap').text
        #href = element.find_element(By.XPATH, "./a["href"]")

        # Extract discount if available
        # could not read somehow 
        #----- TODO: --------------------
        try:
            product_discount = element.find_element(By.CLASS_NAME, 'S-product-item__discount-label notranslate discount-label_discount normal-price-ctn__discount-label').text
        except NoSuchElementException:
            product_discount = 0
            
        # Click on the product to load the reviews
        # -------- TODO: jump from the main page to product page --------
        #new_url = element.find_element(By.CSS_SELECTOR, "div.S-product-item__name >a").get_attribute('href')
        #driver.execute_script("window.open('" + new_url +"');")
        new_url = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.LINK_TEXT, product_name)))
        new_url.click()

        # Scroll to bottom
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        # Wait for the reviews to load
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'common-reviews__list j-expose__common-reviews__list')))
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'sui-icons icons-more_right')))
        
        # Extract the average rating, tags & top 3 reviews
        average_rating = driver.find_element(By.XPATH, '//div[@class="rate-num"]').text
        fit_ratings = driver.find_elements(By.XPATH, '//div[@class="fit-item"]')
        fit_rate = [fit_rating.text for fit_rating in fit_ratings]
        num_review = driver.find_element(By.CLASS_NAME, 'common-reviews__tab-item_active').text

        #product_tags = driver.find_element(By.CLASS_NAME, 'common-reviews__tag').text
        product_tags = driver.find_elements(By.CLASS_NAME, 'common-reviews__tag')
        product_tag_list = [product_tag.text for product_tag in product_tags]
        
        # Grab reviews
        reviews_elems = driver.find_elements(By.CLASS_NAME, 'rate-des')
        top_3_reviews = [review_elem.text for review_elem in reviews_elems]

        # Go back to the product list page
        #driver.execute_script("window.history.go(-1)")
        driver.back()
    
        # Update data
        products_data.append({
        'product_name': product_name,
        'product_price': product_price,
        'product_discount': product_discount,
        'average_rating': average_rating,
        'fit_rating': fit_rate,
        'num_review': num_review,
        'product_tags': product_tag_list,
        'top_3_reviews': top_3_reviews
    })
    
    print(products_data)

    driver.quit()
    return products_data

In [31]:
if __name__ == "__main__":
    url = "https://us.shein.com/style/Women-Clothing-sc-001121425.html?ici=us_tab01navbar05&src_module=topcat&src_tab_page_id=page_home1690914418587&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3D0%60oc%3D0%60ps%3Dtab01navbar05%60jc%3DitemPicking_001121425&srctype=category&userpath=category-CLOTHING&sort=7"
    scrape_shein_products(url)

NameError: name 'StaleElementReferenceException' is not defined

In [49]:
################################
# connection between main page and product page
################################

if __name__ == "__main__":
    url = "https://us.shein.com/style/Women-Clothing-sc-001121425.html?ici=us_tab01navbar05&src_module=topcat&src_tab_page_id=page_home1690914418587&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3D0%60oc%3D0%60ps%3Dtab01navbar05%60jc%3DitemPicking_001121425&srctype=category&userpath=category-CLOTHING&sort=7"
    driver = webdriver.Chrome()  # Replace with your browser driver (e.g., ChromeDriver)
    driver.get(url)
    #WebDriverWait(driver, 10)
    # Wait for the pop-up window to load
    WebDriverWait(driver, 8).until(EC.visibility_of_all_elements_located((By.XPATH,'//i[@class="iconfont icon-close she-close"]')))

    # Click button(close pop-up window)
    close_button = driver.find_element(By.XPATH,'//i[@class="iconfont icon-close she-close"]')
    close_button.click()
    WebDriverWait(driver, 10)
    
    elements = driver.find_element(By.XPATH, "//section[@role='listitem']")

    new_url = elements.find_element(By.CSS_SELECTOR, "div.S-product-item__name >a").get_attribute('href')
    driver.execute_script("window.open('" + new_url +"');")

    #not work(click href)
    #elements.find_element(By.CSS_SELECTOR, "div.S-product-item__name >a").get_attribute('href').click()
    
    #WebDriverWait(driver, 8).until(EC.visibility_of_all_elements_located((By.XPATH,'//h1[@class="product-intro__head-name"]')))
    # Scroll to bottom
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    WebDriverWait(driver, 8)
    driver.quit()
    #print("https://us.shein.com"+href)
    #scrape_shein_products(url)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h1[@class="product-intro__head-name"]"}
  (Session info: chrome=115.0.5790.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010555b6b8 chromedriver + 4937400
1   chromedriver                        0x0000000105552b73 chromedriver + 4901747
2   chromedriver                        0x0000000105110616 chromedriver + 435734
3   chromedriver                        0x0000000105153e0f chromedriver + 712207
4   chromedriver                        0x00000001051540a1 chromedriver + 712865
5   chromedriver                        0x00000001051959a4 chromedriver + 981412
6   chromedriver                        0x000000010517803d chromedriver + 860221
7   chromedriver                        0x0000000105192e76 chromedriver + 970358
8   

In [6]:
    ###############################################
    # GOOOOOOOD TO GO !!!!!!!!! (for product page)
    ###############################################
    
    url = "https://us.shein.com/Button-Front-Denim-Skirt-p-18426903-cat-1937.html?src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3D0%60oc%3D0%60ps%3Dtab01navbar05%60jc%3DitemPicking_001121425&src_module=topcat&src_tab_page_id=page_home1690914418587&mallCode=1"
    driver = webdriver.Chrome()  # Replace with your browser driver (e.g., ChromeDriver)
    driver.get(url)
    WebDriverWait(driver, 10)
    
    # Wait for the pop-up window to load
    WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH,'//i[@class="iconfont icon-close she-close"]')))

    # Click button(close pop-up window)
    close_button = driver.find_element(By.XPATH,'//i[@class="iconfont icon-close she-close"]')
    close_button.click()
    
    # Scroll to bottom
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    # Wait for 10 seconds for the elements to load
    WebDriverWait(driver, 10)

    # Extract product name, price, discount, average rating, tags, and top 100 reviews
    product_name = element.find_element(By.XPATH, '//h1[class="product-intro__head-name"]').text
    
    print("GOT " + product_name + "!")
    
    try:
        product_price = element.find_element(By.CLASS_NAME, 'original from').text
    except NoSuchElementException:
        product_price = element.find_element(By.CLASS_NAME, 'discount from').text
    #href = element.find_element(By.XPATH, "./a["href"]")

    # Extract discount if available
    # could not read somehow 
    #----- TODO: --------------------
    try:
        product_discount = element.find_element(By.CLASS_NAME, 'discount-label').text
    except NoSuchElementException:
        product_discount = 0
            
    average_rating = driver.find_element(By.XPATH, '//div[@class="rate-num"]').text
    fit_ratings = driver.find_elements(By.XPATH, '//div[@class="fit-item"]')
    fit_rate = [fit_rating.text for fit_rating in fit_ratings]
    num_review = driver.find_element(By.CLASS_NAME, 'common-reviews__tab-item_active').text
    #product_tags = driver.find_element(By.CLASS_NAME, 'common-reviews__tag').text
    product_tags = driver.find_elements(By.CLASS_NAME, 'common-reviews__tag')
    product_tag_list = [product_tag.text for product_tag in product_tags]    
        
    # Grab reviews
    reviews_elems = driver.find_elements(By.CLASS_NAME, 'rate-des')
    top_3_reviews = [review_elem.text for review_elem in reviews_elems]

    products_data = []
    products_data.append({
        'product_name': product_name,
        'product_price': product_price,
        'product_discount': product_discount,
        'average_rating': average_rating,
        'fit_rating': fit_rate,
        'num_review': num_review,
        'product_tags': product_tag_list,
        'top_3_reviews': top_3_reviews
    })
    
    print(products_data)

    driver.quit()

[{'average_rating': '4.81', 'fit_rating': ['Small\n7%', 'True to Size\n86%', 'Large\n7%'], 'num_review': 'Customer Reviews ( 1000+ )', 'product_tags': ['will repurchase (8)sexy (86)No noise (11)love (500+)great service (7)trendy (50)fit well (500+)good fabric material (100+)good quality (500+)wrong size (18)'], 'top_3_reviews': ['I absolutely love this skirt. The sound the buttons make while walking may be a little obnoxious, but it just looks so cute!', 'This might be my favorite item from shein! I absolutely love this skirt, it\'s so versatile. The material is thick and not cheap at all. Perfect for all seasons. For reference I\'m 5\'10" tall, 175 lbs, hips 45" and waist 30". The waist was big but this a problem I always have so I had it adjusted. I got a size large.', 'I first purchased this in an XS which was too tight so this is my second time getting it in a S and it’s perfect! I love the versatility with the skirt and I cannot wait to wear it! Make sure to pay attention to the m